In [2]:
from nltk.corpus import wordnet
import nltk
import MeCab
import unicodedata
import pandas as pd

traindf=pd.read_csv("data/train.csv")
testdf=pd.read_csv("data/test.csv")
from nltk.corpus import wordnet
import nltk
import MeCab
import unicodedata
import pandas as pd

# traindf=pd.read_csv("data/train.csv")
# testdf=pd.read_csv("data/test.csv")

import mojimoji
import re
import jaconv



def parse_text(text, debug=False):
    '''
    Get location
    '''

#     text = mojimoji.zen_to_han(text, kana=False)
    text = re.sub(r'［[^］]+］', '', text)  
#     text = re.sub(r'[\.*[\]]', '', text)    
#     text = re.sub(r'[\.*[\]]', '', text)
#     text = re.sub(r'', '', text)    
#     text = re.sub(r'[\(（[].*[）\)]]', '', text)
    text = re.sub(r'[\s、]', '', text)
    text = re.sub(r'一', '', text)
#     text = re.sub(r'…', '', text)  
#     text = re.sub(r'/＼', '', text)
    text = re.sub(r'[0-9]', '0', text)


#     text = jaconv.kata2hira(text)
    return text

# df=pd.concat([traindf,testdf])
# df.body=df.body.map(parse_text)
# traindf.body=traindf.body.map(parse_text)
# testdf.body=testdf.body.map(parse_text)


def normalize(text):
    normalized_text = normalize_unicode(text)
    normalized_text = normalize_number(normalized_text)
    normalized_text = lower_text(normalized_text)
    return normalized_text


def lower_text(text):
    return text.lower()


def normalize_unicode(text, form='NFKC'):
    normalized_text = unicodedata.normalize(form, text)
    return normalized_text


def lemmatize_term(term, pos=None):
    if pos is None:
        synsets = wordnet.synsets(term)
        if not synsets:
            return term
        pos = synsets[0].pos()
        if pos == wordnet.ADJ_SAT:
            pos = wordnet.ADJ
    return nltk.WordNetLemmatizer().lemmatize(term, pos=pos)


def normalize_number(text):
    """
    pattern = r'\d+'
    replacer = re.compile(pattern)
    result = replacer.sub('0', text)
    """
    # 連続した数字を0で置換
    replaced_text = re.sub(r'\d+', '0', text)
    return replaced_text


# mecab = MeCab.Tagger('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd/')
# mecab=MeCab.Tagger ('-d /usr/local/lib/mecab/dic/UniDic-kindai_1603')
mecab = MeCab.Tagger('-Owakati')

# 形態素解析をして、名詞だけ取り出す


def tokenize(text):
#     available_norm = ['接尾', '一般', '形容動詞語幹', 'サ変接続']
    node = mecab.parseToNode(text)
    l = []
    while node:
        l.append(node.surface)
        node = node.next
    return l


# 記事群のdictについて、形態素解析をしてリストに返す
def get_words(contents):
    available_norm = ['接尾', '一般', '形容動詞語幹', 'サ変接続']
    node = mecab.parseToNode(contents)
    l = []
    while node:

        l.append(node.surface)
        node = node.next
    return l

# 一つの記事を形態素解析して返す


def get_words_main(content):
    return [token for token in tokenize1(content)]


def get_words_main1(content):
    retoken = ""
    for token in tokenize(content):
        retoken += token+" "
    return retoken

def tokenize1(text):
#     available_norm = ['接尾', '一般', '形容動詞語幹', 'サ変接続']
    node = mecab.parseToNode(text)
    l = []
    while node:
        l.append(node.surface)
        node = node.next
    return ' '.join(l)
df = pd.concat([traindf, testdf])
df.body = df.body.map(parse_text).map(tokenize1)
#     count = CountVectorizer(analyzer=tokenize)
#     bags = count.fit_transform(df.body.values)
#     # print(bags.toarray())

#     features = count.get_feature_names()
#     # # print(features)

#     bodyvec = pd.DataFrame(bags.toarray(), columns=features)
#     newdf = pd.concat([df.reset_index(drop=True), pd.DataFrame(bodyvec)], axis=1)
train = df.dropna().drop(["writing_id", ], axis=1)
test = df[df.author.isnull()].drop(["writing_id"], axis=1)
test = test.drop(["author"], axis=1)
X = train.drop(["author"], axis=1)
y = train.author

/Users/ogata2/.pyenv/versions/3.7.5/lib/python3.7/site-packages/ipykernel_launcher.py:138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [61]:
# from sklearn.feature_extraction.text import CountVectorizer


# def preprocess():
#     traindf = pd.read_csv("data/train.csv")
#     testdf = pd.read_csv("data/test.csv")

#     df = pd.concat([traindf, testdf])
#     df.body = df.body.map(parse_text)
#     count = CountVectorizer(analyzer=tokenize)
#     bags = count.fit_transform(df.body.values)
#     # print(bags.toarray())

#     features = count.get_feature_names()
#     # # print(features)

#     bodyvec = pd.DataFrame(bags.toarray(), columns=features)
#     newdf = pd.concat([df.reset_index(drop=True), pd.DataFrame(bodyvec)], axis=1)
#     train = newdf.dropna().drop(["writing_id", "body", ], axis=1)
#     test = newdf[newdf.author.isnull()].drop(["writing_id", "body"], axis=1)
#     test = test.drop(["author"], axis=1)
#     X = train.drop(["author"], axis=1)
#     y = train.author
#     return X, y, test


# # X, y, test = preprocess()

In [64]:
def preprocess1():
    traindf = pd.read_csv("data/train.csv")
    testdf = pd.read_csv("data/test.csv")

    df = pd.concat([traindf, testdf])
    df.body = df.body.map(parse_text).map(tokenize1)
#     count = CountVectorizer(analyzer=tokenize)
#     bags = count.fit_transform(df.body.values)
#     # print(bags.toarray())

#     features = count.get_feature_names()
#     # # print(features)

#     bodyvec = pd.DataFrame(bags.toarray(), columns=features)
#     newdf = pd.concat([df.reset_index(drop=True), pd.DataFrame(bodyvec)], axis=1)
    train = df.dropna().drop(["writing_id", ], axis=1)
    test = df[df.author.isnull()].drop(["writing_id"], axis=1)
    test = test.drop(["author"], axis=1)
    X = train.drop(["author"], axis=1)
    y = train.author
    return X, y, test
X, y, test=preprocess1()

In [68]:
X

,body
0,先ごろ の 本欄 に 僕 の 「 風 報 」 に かい た 「 天皇陛下 に 捧ぐる 言葉...
1,旅 の 眼 に 映じ た 外国 の 正月 を といふ お 需 め で 昔 前 の 記憶 か...
2,或 る 心持 の よい 夕方 日比谷公園 の 樹 の 繁み の 間 で 若葉 楓 の 梢 ...
3,島々 《 しま ／ ＼ 》 と 云 ふ 町 の 宿屋 へ 着い た の は 午 過ぎ ――...
4,或 る 田舎 に 母 と 子 と が 住ん で い た 。 そして 或 る 年 の 秋 次...
...,...
3307,八 五 郎 の 取柄 は 誰 と で も すぐ 友達 に なれる こと でし た 。 長 ...
3308,ある 婦人 が 私 に 言 つ た 。 私 が 情痴 作家 など ゝ 言 は れる こと ...
3309,諸君 は 東京 市 某 町 某 番地 なる 風 博士 の 邸宅 を 御存じ で あろ う ...
3310,「 御免 」 少し 職業 的 に 落着き 拂 つた 聲錢 形 平次 は それ を 聞く と...


In [69]:
list(X)

['body']

In [70]:
X.body

0        先ごろ の 本欄 に 僕 の 「 風 報 」 に かい た 「 天皇陛下 に 捧ぐる 言葉...
1        旅 の 眼 に 映じ た 外国 の 正月 を といふ お 需 め で 昔 前 の 記憶 か...
2        或 る 心持 の よい 夕方 日比谷公園 の 樹 の 繁み の 間 で 若葉 楓 の 梢 ...
3        島々 《 しま ／ ＼ 》 と 云 ふ 町 の 宿屋 へ 着い た の は 午 過ぎ ――...
4        或 る 田舎 に 母 と 子 と が 住ん で い た 。 そして 或 る 年 の 秋 次...
                              ...                        
3307     八 五 郎 の 取柄 は 誰 と で も すぐ 友達 に なれる こと でし た 。 長 ...
3308     ある 婦人 が 私 に 言 つ た 。 私 が 情痴 作家 など ゝ 言 は れる こと ...
3309     諸君 は 東京 市 某 町 某 番地 なる 風 博士 の 邸宅 を 御存じ で あろ う ...
3310     「 御免 」 少し 職業 的 に 落着き 拂 つた 聲錢 形 平次 は それ を 聞く と...
3311     ちやう ど 今日 （ 十月 三 日 ） 文部省 で 著作 家 側 を 招い て 新 カナヅ...
Name: body, Length: 3312, dtype: object

In [19]:
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

train_X, test_X, train_y, test_y = train_test_split(
    X, y, test_size=0.2, random_state=0)  # 80%のデータを学習データに、20%を検証データにする

In [20]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()  # ロジスティック回帰モデルのインスタンスを作成
lr.fit(train_X, train_y)  # ロジスティック回帰モデルの重みを学習
pred = lr.predict(test_X)
print('confusion matrix = \n', confusion_matrix(y_true=test_y, y_pred=pred))
print('accuracy = ', accuracy_score(y_true=test_y, y_pred=pred))
print('precision = ', precision_score(y_true=test_y, y_pred=pred))
print('recall = ', recall_score(y_true=test_y, y_pred=pred))
print('f1 score = ', f1_score(y_true=test_y, y_pred=pred))

confusion matrix = 
 [[606   2]
 [  7  48]]
accuracy =  0.9864253393665159
precision =  0.96
recall =  0.8727272727272727
f1 score =  0.9142857142857144


In [21]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

mlp = MLPClassifier(
    **{"hidden_layer_sizes": (128, 128, 128, 128, 128, 128), "random_state": 42})
mlp.fit(train_X, train_y)
pred = mlp.predict(test_X)
print('confusion matrix = \n', confusion_matrix(y_true=test_y, y_pred=pred))
print('accuracy = ', accuracy_score(y_true=test_y, y_pred=pred))
print('precision = ', precision_score(y_true=test_y, y_pred=pred))
print('recall = ', recall_score(y_true=test_y, y_pred=pred))
print('f1 score = ', f1_score(y_true=test_y, y_pred=pred))

confusion matrix = 
 [[605   3]
 [  3  52]]
accuracy =  0.9909502262443439
precision =  0.9454545454545454
recall =  0.9454545454545454
f1 score =  0.9454545454545454


In [98]:
mlp = MLPClassifier(
    **{"hidden_layer_sizes": (128, 128, 128, 128, 128, 128), "random_state": 42})
mlp.fit(X, y)
pred = mlp.predict(test)
# pred = model.predict(np.array(test))
pred = np.where(pred > 0.5, 1, 0)
sub = pd.DataFrame(pd.read_csv("data/test.csv")['writing_id'])
sub["author"] = list(pred)
sub.to_csv("submission.csv", index=False)

In [25]:
!pip install tensorflow

  Using cached https://files.pythonhosted.org/packages/2c/72/6b3264aa2889b7dde7663464b99587d95cd6a5f3b9b30181f14d78a63e64/tensorflow-2.0.0-cp37-cp37m-macosx_10_11_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/8a/48/a76be51647d0eb9f10e2a4511bf3ffb8cc1e6b14e9e4fab46173aa79f981/termcolor-1.1.0.tar.gz
  Using cached https://files.pythonhosted.org/packages/fc/08/8b927337b7019c374719145d1dceba21a8bb909b93b1ad6f8fb7d22c1ca1/tensorflow_estimator-2.0.1-py2.py3-none-any.whl
     |████████████████████████████████| 3.8MB 1.2MB/s eta 0:00:01     |████████▋                       | 1.0MB 1.5MB/s eta 0:00:02     |█████████████▏                  | 1.6MB 1.5MB/s eta 0:00:02
  Using cached https://files.pythonhosted.org/packages/b8/83/755bd5324777875e9dff19c2e59daec837d0378c09196634524a3d7269ac/opt_einsum-3.1.0.tar.gz
  Using cached https://files.pythonhosted.org/packages/00/83/b4a77d044e78ad1a45610eb88f745be2fd2c6d658f9798a15e384b7d57c9/wheel-0.33.6-py2.py3-none-any.whl
    Running s

In [33]:
test

,body
0,"[, 夕方, 降り出し, た, 雨, は, その, 晩, 遅く, まで, 続い, た, 。,..."
1,"[, この, 「, 東北, 文学, 」, という, 雑誌, の, 貴重, な, 紙面, の,..."
2,"[, 幼少, の, ころ, 高知, 《, こうち, 》, の, 城下, から, 東, に, ..."
3,"[, ○, 「, 三, 人, 姉妹, 」, で, マーシャ, が, どんな, 風, に, 活..."
4,"[, 眼鏡, 或, 日, 趣味, に関して, 人, に, 問, はれ, た, 。, 稍, 暫..."
...,...
1415,"[, 伝統, 的, な, 女形, と, 云う, もの, の, 型, に, 嵌っ, て, 終始..."
1416,"[, 「, あ, ツヒ, 人殺し, ツ, 」, 宵闇, を, 劈, 《, つん, ざ, 》,..."
1417,"[, 本誌, （, 「, 劇作, 」, ）, 三月, 号, に, 発表, さ, れ, た, ..."
1418,"[, 昔, 《, むかし, 》, は, いま, より, も, もっと, 松, 《, まつ, ..."


In [3]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(pd.concat([X.body,test.body]))
x_train = tokenizer.texts_to_sequences(X.body)
x_test = tokenizer.texts_to_sequences(test.body)


,body
0,先ごろ の 本欄 に 僕 の 「 風 報 」 に かい た 「 天皇陛下 に 捧ぐる 言葉...
1,旅 の 眼 に 映じ た 外国 の 正月 を といふ お 需 め で 昔 前 の 記憶 か...
2,或 る 心持 の よい 夕方 日比谷公園 の 樹 の 繁み の 間 で 若葉 楓 の 梢 ...
3,島々 《 しま ／ ＼ 》 と 云 ふ 町 の 宿屋 へ 着い た の は 午 過ぎ ――...
4,或 る 田舎 に 母 と 子 と が 住ん で い た 。 そして 或 る 年 の 秋 次...
...,...
3307,八 五 郎 の 取柄 は 誰 と で も すぐ 友達 に なれる こと でし た 。 長 ...
3308,ある 婦人 が 私 に 言 つ た 。 私 が 情痴 作家 など ゝ 言 は れる こと ...
3309,諸君 は 東京 市 某 町 某 番地 なる 風 博士 の 邸宅 を 御存じ で あろ う ...
3310,「 御免 」 少し 職業 的 に 落着き 拂 つた 聲錢 形 平次 は それ を 聞く と...


In [59]:
X.head(3)

,body
0,先ごろ の 本欄 に 僕 の 「 風 報 」 に かい た 「 天皇陛下 に 捧ぐる 言葉...
1,旅 の 眼 に 映じ た 外国 の 正月 を といふ お 需 め で 昔 前 の 記憶 か...
2,或 る 心持 の よい 夕方 日比谷公園 の 樹 の 繁み の 間 で 若葉 楓 の 梢 ...


In [4]:
for text, vector in zip(X.body.head(3), x_train[0:3]):
    print(text)
    print(vector)

 先ごろ の 本欄 に 僕 の 「 風 報 」 に かい た 「 天皇陛下 に 捧ぐる 言葉 」 を 評し て 俗 うけ を 狙っ た 媚態 露出 だ と の こと で ある が 白井 明 先生 の 鑑賞 眼 は 浅薄 低俗 と 申さ なけれ ば なら ない 。 あの 文章 に こもる 祖国 へ よせる 僕 の 愛情 や あれ を 書か ず に い られ なかっ た 情熱 を 読みとる こと が でき ない と は 白井 先生 が 頃日 書く 意味 も ない 駄文 ばかり 書い てる せい な の で ある 。 いったい に 文学 の 反語 性 に 味読 の 及ば ぬ 識見 低俗 な ヤカラ が 文学 を 批評 する という の が 間違っ て いる 。 僕 の 「 堕落 論 」 その他 の エッセイ に し て も 小説 に し て も その 反語 に こもる 正しい 意味 を 理解 し 得 ず に 軽率 な 判読 断定 を 下す から 読者 に 誤読 の お手本 を 与え て いる よう な もの で ある 。 本名 で は 愚かしい ソラゴト しか 書け ず 匿名 で しか 本音 の 吐け ぬ 文学 者 など という もの は ない 。 僕 に は 匿名 の 必要 は ない 。 いつ でも 本音 を 吐き ギリギリ の こと を 言っ てる から だ 。 だから また ぼく の 本音 は 文学 の 本質 的 な もの で あり 単なる 中傷 の ケチ くさい 汚らし さ は ない の で ある 。 白井 明 先生 も 本名 で 本音 を 吐く こと を 学び た ま え 。 本名 で 君 の ケチアサマシサ を さらけだす こと の 苦痛 に 堪え て その 争い の 嵐 の 中 で 自分 を 育て た ま え 。 さ すれ ば 文学 の 本質 に も やがて 近づき うる で あろ う 。 「 天皇陛下 に さ ゝ ぐる 言葉 」 に こもる 大いなる 愛情 も やみ がたい 情熱 も 君 の 目 に は 逆 の 意味 に うつる の も きわめて 当然 な こと で ある 。 しかし こういう 愚 に も つか ない 批評 で も それ が 君 の 本音 なら 仕方 が ない から せめて 本名 で 書か れん こと を 。 さ すれ ば 進歩 は あり 

In [5]:
max_len=1000
x_train = pad_sequences(x_train, maxlen=max_len)
x_test = pad_sequences(x_test, maxlen=max_len)

In [12]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding
from sklearn.metrics import f1_score
from keras.callbacks import Callback

class F1Callback(Callback):
    def __init__(self, model, X_val, y_val):
        self.model = model
        self.X_val = X_val
        self.y_val = y_val

    def on_epoch_end(self, epoch, logs):
        pred = self.model.predict(self.X_val)
        f1_val = f1_score(self.y_val, np.round(pred))
        print("f1_val =", f1_val)
        # 以下チェックポイントなど必要なら書く

vocabulary_size = len(tokenizer.word_index) + 1  # 学習データの語彙数+1

model = Sequential()

model.add(Embedding(input_dim=vocabulary_size, output_dim=32))
model.add(LSTM(16, return_sequences=False))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 32)          4722304   
_________________________________________________________________
lstm_2 (LSTM)                (None, 16)                3136      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 4,725,457
Trainable params: 4,725,457
Non-trainable params: 0
_________________________________________________________________


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pandas as pd
import warnings
# display(HTML("<style>.container { width:80% !important; }</style>"))
warnings.filterwarnings('ignore')
train_X, valid_X, train_y, valid_y = train_test_split(x_train, y, test_size = 0.2, random_state = 0) # 80%のデータを学習データに、20%を検証データにする


In [15]:
history = model.fit(
    train_X, train_y, batch_size=32, epochs=10,
    validation_data=(valid_X, valid_y),callbacks=[F1Callback(model, valid_X, valid_y)]
)

Train on 2649 samples, validate on 663 samples
Epoch 1/10
2649/2649 [==============================] - 47s 18ms/step - loss: 0.3708 - accuracy: 0.9226 - val_loss: 0.2541 - val_accuracy: 0.9170
f1_val = 0.0
Epoch 2/10
2649/2649 [==============================] - 45s 17ms/step - loss: 0.1798 - accuracy: 0.9320 - val_loss: 0.1833 - val_accuracy: 0.9231
f1_val = 0.13559322033898305
Epoch 3/10
2649/2649 [==============================] - 50s 19ms/step - loss: 0.1346 - accuracy: 0.9577 - val_loss: 0.1347 - val_accuracy: 0.9608
f1_val = 0.723404255319149
Epoch 4/10
2649/2649 [==============================] - 57s 22ms/step - loss: 0.0772 - accuracy: 0.9758 - val_loss: 0.1128 - val_accuracy: 0.9593
f1_val = 0.7216494845360825
Epoch 5/10
2649/2649 [==============================] - 57s 21ms/step - loss: 0.0490 - accuracy: 0.9875 - val_loss: 0.1028 - val_accuracy: 0.9638
f1_val = 0.7551020408163266
Epoch 6/10
2649/2649 [==============================] - 61s 23ms/step - loss: 0.0329 - accuracy: 0

KeyboardInterrupt: 

In [10]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
pred = model.predict_classes(valid_X)
print('confusion matrix = \n', confusion_matrix(y_true=valid_y, y_pred=pred))
print('accuracy = ', accuracy_score(y_true=valid_y, y_pred=pred))
print('precision = ', precision_score(y_true=valid_y, y_pred=pred))
print('recall = ', recall_score(y_true=valid_y, y_pred=pred))
print('f1 score = ', f1_score(y_true=valid_y, y_pred=pred))

confusion matrix = 
 [[604   4]
 [ 18  37]]
accuracy =  0.9668174962292609
precision =  0.9024390243902439
recall =  0.6727272727272727
f1 score =  0.7708333333333333
